In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import modules.eda as Detective
import pandas as pd
import warnings
import numpy as np
import random

warnings.filterwarnings('ignore')

In [4]:
reviews = pd.read_csv("./data/normalize_reviews.csv").fillna("")
reviews = reviews[['raw_comment', 'normalize_comment', 'emoji', 'label']]

reviews.head()

,raw_comment,normalize_comment,emoji,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhd...,giao hàng không đúng cần phê bình,,0
1,Chất lượng sản phẩm tạm được. Giao...,chất lượng sản phẩm tạm được giao ...,,0
2,Ko có lắc tay như hình,không có lắc tay như hình,,0
3,Giao hàng lâu. Bảo có lắc tay mà k thâ...,giao hàng lâu bảo có lắc tay mà không ...,,0
4,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,😢,0


Hiện tại, dataset chúng ta có hai feature là `normalize_comment` và `emoji`, chúng ta có thể xây dựng một sentiment analysis classifier bằng hai cách:
  * **Cách 1**: Chúng ta có thể đem feature `emoji` để thêm vào phần đầu hoặc cuối của feature `normalize_comment`. Sau đó tiến hành đào tạo một model duy nhất với input là feature `normalize_comment` đã được thêm vào feature `emoji`.
    * Ưu điểm: Quá trình đào tạo model đỡ tốn thời gian hơn do ta chỉ thực hiện build một model duy nhất.
    * Nhược điểm:
      * Có khả năng làm suy giảm sức mạnh của model, nhiều người họ còn xây dưng riêng một **emoji classifier** bởi vì theo họ nhiều khi emoji đó mang lại ý nghĩa thậm chí còn cao hơn so với ngữ nghĩa của comment đó. Trong khi việc đào tạo một emoji classifier đỡ vất vã hơn.
      * Ta phải tốn thời gian vào việc xây dựng thêm một model riêng sau đó lại tổng hợp kết quả của emoji classifier model này với sentiment classifier model ban đầu.
  * **Cách 2**: Ta có thể xây dựng hai model riêng biệt, một sentiment analysis model đảm nhận nhiệm vụ phân lớp cho câu chữ và một emoji analysis model đảm nhận nhiệm vụ phân lớp trên emoji. Sau đó ta kết hợp hai model này để cho ra kết quả phân lớp cuối cùng.
    * Ưu điểm: Kết quả phân lớp cho ra có khả năng chính xác hơn, ta có thể linh hoạt trong việc thiếp lập trọng số để ưu tiên giá trị của model nào mang lại lợi ích cao hơn cho kết quả ở bước kết hợp hai model lại để dự đoán.
    * Nhược điểm:
      * Vất vả hơn về mặt tiền xử lí dữ liệu, tốn thêm thời gian phân tích và thiết kế thêm một emoji classifier.
      * Phần lớn trong dataset hiện tại, số comment chứa emoji không nhiều. Như project 2 ta đã thấy toàn bộ dataset của ta chỉ khoảng 1300 comment là có chứa emoji kèm theo, ta có thể khắc phục bằng một trong những cách:
        * Crawl thêm data, nhưng việc này không khả khi. Ta đã đề cập rằng số comment chứa emoji rất ít trong một dataset nên việc ta hi sinh thời gian chỉ để trích xuất một phần nhỏ trong một dataset cực lớn sau khi crawl về có thể phí phạm.
        * Sử dụng kĩ thuật **Data Upsampler** với package của IBM `from imblearn.over_sampling import RandomOverSampler`.

    $\Rightarrow$ Hướng giải quyết: Quả thật ta có khoảng 1300 observer thì là một con số không quá ít cũng không quá nhiều, số lượng emoji chứa trong một comment cũng không nhiều (không tính các duplicate emoji). Bài toán của chúng ta đơn thuần chỉ là phân tích các emoji ra hai class là negative và positive. Hãy cùng nhìn lại một dataset nổi tiếng khác là **Iris**, ta có 3 class ở target variable và 150 observe. Vậy ta vẫn có thể build một classifier model đủ tốt nếu input của ta cũng **đủ tốt**.

# Emoji sentiment analysis

Như vậy chiến lược đào tạo một Emoji sentiment analysis của chúng ta sẽ như sau:
  * Bước 1: Tiến hành lấy toàn bộ các emoji trong các comment và lưu trong cấu trúc dữ liệu `set`, mục tiêu ta cần biết có bao nhiêu unique emoji trong dataset của chúng ta.
  * Bước 2: Tạo 2 dictionary, một dictionary có key là emoji và value là index của emoji đó trong `set`, dictionary thứ hai có key là index của emoji trong `set` và value là emoji.
  * Bước 3: Giả sử có $n$ unique emoji trong dataset, ta sẽ chỉ quan tâm đến các comment chứa emoji, và với từng comment như vậy ta tạo một vector $n$ phần tử sau đó áp dụng phương pháp **Bag of words** lên vector này.
  * Bước 4: Xem xét $n$ có lớn không, nếu quá lớn thì có thể áp dụng các phương pháp giảm chiều dữ liệu.
  * Bước 5: Dùng các vector $n$ phần tử + label của chúng sau khi đã qua bước 4 để đào tạo một classifier model.